<a href="https://colab.research.google.com/github/rohitkeshri803/Breast-Cancer-Detection/blob/main/BreastCancerDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [4]:
# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [5]:
# Load data
data = load_breast_cancer()
X = data.data
y = data.target

In [6]:
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
# Convert full dataset to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

In [9]:
# Neural Net Class (No Sigmoid, BCEWithLogitsLoss will handle that)
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [10]:
# Hyperparameter grid
hidden_sizes = [32, 64]
learning_rates = [0.001, 0.01]
num_epochs = 50
k_folds = 5


In [11]:
# Cross-validation setup
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
input_size = X.shape[1]

In [12]:
best_config = None
best_val_acc = 0.0

In [13]:
# Grid Search with Cross-Validation
for hidden_size in hidden_sizes:
    for lr in learning_rates:
        fold_accuracies = []

        print(f"\n🔍 Trying config: hidden_size={hidden_size}, lr={lr}")

        for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
            model = NeuralNet(input_size, hidden_size).to(device)
            optimizer = optim.Adam(model.parameters(), lr=lr)
            criterion = nn.BCEWithLogitsLoss()

            # Data split
            X_train = X_tensor[train_idx].to(device)
            y_train = y_tensor[train_idx].to(device)
            X_val = X_tensor[val_idx].to(device)
            y_val = y_tensor[val_idx].to(device)

            # Training loop
            for epoch in range(num_epochs):
                model.train()
                optimizer.zero_grad()
                outputs = model(X_train)
                loss = criterion(outputs, y_train.view(-1,1))
                loss.backward()
                optimizer.step()

            # Evaluation
            model.eval()
            with torch.no_grad():
                val_outputs = model(X_val)
                val_probs = torch.sigmoid(val_outputs).cpu().numpy().flatten()
                val_preds = np.round(val_probs)
                val_labels = y_val.cpu().numpy()

                acc = accuracy_score(val_labels, val_preds)
                fold_accuracies.append(acc)

            print(f"  Fold {fold+1}/{k_folds}: Accuracy = {acc:.4f}")

        avg_acc = np.mean(fold_accuracies)
        print(f"📊 Avg Accuracy for config hidden_size={hidden_size}, lr={lr} = {avg_acc:.4f}")

        # Keep track of best
        if avg_acc > best_val_acc:
            best_val_acc = avg_acc
            best_config = {'hidden_size': hidden_size, 'learning_rate': lr}


🔍 Trying config: hidden_size=32, lr=0.001
  Fold 1/5: Accuracy = 0.9737
  Fold 2/5: Accuracy = 0.9035
  Fold 3/5: Accuracy = 0.9561
  Fold 4/5: Accuracy = 0.9211
  Fold 5/5: Accuracy = 0.9204
📊 Avg Accuracy for config hidden_size=32, lr=0.001 = 0.9349

🔍 Trying config: hidden_size=32, lr=0.01
  Fold 1/5: Accuracy = 0.9825
  Fold 2/5: Accuracy = 0.9912
  Fold 3/5: Accuracy = 0.9649
  Fold 4/5: Accuracy = 0.9912
  Fold 5/5: Accuracy = 0.9646
📊 Avg Accuracy for config hidden_size=32, lr=0.01 = 0.9789

🔍 Trying config: hidden_size=64, lr=0.001
  Fold 1/5: Accuracy = 0.9649
  Fold 2/5: Accuracy = 0.9298
  Fold 3/5: Accuracy = 0.9561
  Fold 4/5: Accuracy = 0.9561
  Fold 5/5: Accuracy = 0.9381
📊 Avg Accuracy for config hidden_size=64, lr=0.001 = 0.9490

🔍 Trying config: hidden_size=64, lr=0.01
  Fold 1/5: Accuracy = 0.9825
  Fold 2/5: Accuracy = 0.9912
  Fold 3/5: Accuracy = 0.9649
  Fold 4/5: Accuracy = 0.9912
  Fold 5/5: Accuracy = 0.9646
📊 Avg Accuracy for config hidden_size=64, lr=0.01 =

In [14]:
print("\n✅ Best Hyperparameter Configuration:")
print(best_config)
print(f"✅ Best Cross-Validated Accuracy: {best_val_acc:.4f}")


✅ Best Hyperparameter Configuration:
{'hidden_size': 32, 'learning_rate': 0.01}
✅ Best Cross-Validated Accuracy: 0.9789
